In [ ]:
from rocketpy import Rocket, Environment, Flight, LiquidMotor, Fluid, CylindricalTank, MassFlowRateBasedTank, MassBasedTank
from math import exp
from datetime import datetime, timedelta
from CoolProp.CoolProp import PropsSI
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#Far-out
tomorrow = datetime.now() + timedelta(days=1)
env = Environment(
    date=tomorrow,  
    latitude=35.35,
    longitude=-117.81, # Far-out not precise
)
env.set_elevation("Open-Elevation")
env.set_atmospheric_model(type="forecast", file="GFS")
env.max_expected_height = 10000
env.plots.atmospheric_model()

# env.info()

In [ ]:
# Stałe dla turbulencji
external_tank_diameter = 0.2
tank_height = 1.13
thickness_tank = 0.005
thickness_piston = 0.01

# Parametry, które potrzebuję
p_0 = 63e5 #ciśnienie początkowe
piston_position = 0.85
total_oxidizer_mass = 18
flux_time = 11.2 # the start time will be useful as well
# csv z masą utleniacza i paliwa w czasie
ethanol_temperature = 300
# Parametry, które zakładem
gas_initial_mass_fuel = 0

# Parametry, które wyliczam
N20_liq_density = PropsSI("D", "P", p_0, "Q", 0, "NitrousOxide")
N20_gas_density = PropsSI("D", "P", p_0, "Q", 1, "NitrousOxide")
ethanol_liq_density = PropsSI("D", "P", p_0-1e5, "T", ethanol_temperature, "Ethanol") # PropSI pokazuje 63e5 powyżej krytycznego, dlatego odejmuję
ethanol_gas_density = PropsSI("D", "P", p_0-1e5, "Q", 1, "Ethanol")

volume_tank = 0.25*np.pi*(external_tank_diameter-2*thickness_tank)**2*tank_height
volume_piston = 0.25*np.pi*(external_tank_diameter-2*thickness_tank)**2*thickness_piston
volume_oxidizer = piston_position*volume_tank
volume_fuel = volume_tank - volume_oxidizer - volume_piston

gas_initial_mass_ox = (volume_oxidizer - (total_oxidizer_mass / N20_liq_density)) / (1/N20_gas_density - 1/N20_liq_density)
liquid_initial_mass_ox = total_oxidizer_mass - gas_initial_mass_ox
liquid_initial_mass_fuel = volume_fuel * ethanol_liq_density

# Parametry stricte pod tank_geometry
tank_radius = (external_tank_diameter - 2*thickness_tank) / 2
adjusted_height_ox = piston_position*tank_height
adjusted_height_fuel = tank_height - adjusted_height_ox - thickness_piston

# Dla sprawdzenia czy wychodzi mi to samo co volume_oxidizer i volume_fuel
fuel_tank_volume = np.pi * tank_radius**2 * adjusted_height_fuel
oxidizer_tank_volume = np.pi * tank_radius**2 * adjusted_height_ox

In [ ]:
# Defining fluids
print(f"N2O liquid density: {N20_liq_density} kg/m^3")
print(f"N2O gas density: {N20_gas_density} kg/m^3")
print(f"Ethanol liquid density: {ethanol_liq_density} kg/m^3")
print(f"Ethanol gas density: {ethanol_gas_density} kg/m^3")

oxidizer_liq = Fluid(name="N2O_l", density=N20_liq_density)
oxidizer_gas = Fluid(name="N2O_g", density=N20_gas_density)
fuel_liq = Fluid(name="ethanol_l", density=ethanol_liq_density) 
fuel_gas = Fluid(name="ethanol_g", density=ethanol_gas_density)

In [ ]:
print(f"Total tank volume: {volume_tank} m^3")
print(f"Volume occupied by the piston: {volume_piston} m^3")
print(f"Volume of oxidizer: {volume_oxidizer} m^3")
print(f"Volume of fuel: {volume_fuel} m^3")
print(f"Initial mass of oxidizer gas: {gas_initial_mass_ox} kg")
print(f"Initial mass of oxidizer liquid: {liquid_initial_mass_ox} kg")
print(f"Fuel mass: {liquid_initial_mass_fuel} kg")
print(f"Adjusted height of oxidizer tank: {adjusted_height_ox} m")
print(f"Adjusted height of fuel tank: {adjusted_height_fuel} m")
print(f"Fuel tank volume: {fuel_tank_volume} m^3")
print(f"Oxidizer tank volume: {oxidizer_tank_volume} m^3")

In [ ]:
# Tank geometry
oxidizer_tank_geometry = CylindricalTank(
    radius=tank_radius,
    height=adjusted_height_ox,
)
fuel_tank_geometry = CylindricalTank(
    radius=tank_radius,
    height=adjusted_height_fuel,
)

In [ ]:
# Define tanks
mass_flow_rate_liq = round(liquid_initial_mass_ox/flux_time, 2) - 0.005 # waiting for csv
mass_flow_rate_gas = round(gas_initial_mass_ox/flux_time - 0.005, 2) # waiting for csv

oxidizer_tank = MassFlowRateBasedTank(
    name="oxidizer tank",
    geometry=oxidizer_tank_geometry,
    flux_time=(1, flux_time+1), 
    initial_liquid_mass=liquid_initial_mass_ox, 
    initial_gas_mass=gas_initial_mass_ox,
    liquid_mass_flow_rate_in=0,
    liquid_mass_flow_rate_out=mass_flow_rate_liq, 
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=mass_flow_rate_gas, 
    liquid=oxidizer_liq,
    gas=oxidizer_gas,
)


In [ ]:
#mass based tank 
gas_mass_function_ox = oxidizer_tank.gas_mass
liq_mass_function_ox = oxidizer_tank.liquid_mass
mass_based_oxidizer_tank = MassBasedTank(
    name="mass based oxidizer tank",
    geometry=oxidizer_tank_geometry,
    flux_time=(1, flux_time+1),
    gas=oxidizer_gas,
    liquid=oxidizer_liq,
    gas_mass=gas_mass_function_ox,
    liquid_mass=liq_mass_function_ox,
    discretize=100,
)


In [ ]:
# oxidizer_tank.all_info()
# mass_based_oxidizer_tank.all_info()

In [ ]:
#Fuel tank
fuel_mass_flow_rate = liquid_initial_mass_fuel / flux_time - 0.01
fuel_tank = MassFlowRateBasedTank(
    name="fuel tank",
    geometry=fuel_tank_geometry,
    flux_time=(1, flux_time+1),
    initial_liquid_mass=liquid_initial_mass_fuel-0.00001, #Same as above, only guess
    initial_gas_mass=gas_initial_mass_fuel,
    liquid_mass_flow_rate_in=0,
    liquid_mass_flow_rate_out=fuel_mass_flow_rate, #heuristics
    gas_mass_flow_rate_in=0,
    gas_mass_flow_rate_out=0,
    liquid=fuel_liq,
    gas=fuel_gas,
)

In [ ]:
#mass based tank
gas_mass_function_fuel = fuel_tank.gas_mass
liq_mass_function_fuel = fuel_tank.liquid_mass
mass_based_fuel_tank = MassBasedTank(
    name="mass based fuel tank",
    geometry=fuel_tank_geometry,
    flux_time=(1, flux_time+1),
    gas=fuel_gas,
    liquid=fuel_liq,
    gas_mass=gas_mass_function_fuel,
    liquid_mass=liq_mass_function_fuel,
    discretize=100,
)

In [ ]:
# fuel_tank.all_info()
# mass_based_fuel_tank.all_info()

In [ ]:
z4000 = LiquidMotor(
    thrust_source=".\\data\\AGH-SS_Z4000-10sBurn-optimal.eng", #From tests
    dry_mass=2.7,
    dry_inertia=(0.02143, 0.02143, 0.005535), #This should be calculated using CAD, here I use estimations
    nozzle_radius=0.036, #From technical report
    center_of_dry_mass_position=0.144, #Estimated from openrocket
    nozzle_position=0,
    burn_time=14.4,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)
z4000.add_tank(tank=oxidizer_tank, position=1.285) #From nozzle to center of the tank
z4000.add_tank(tank=fuel_tank, position=2.01)
# z4000.all_info()

In [ ]:
trb = Rocket(
    radius=0.1,
    mass=58.367, 
    inertia=(75.502, 75.502, 0.43), 
    power_off_drag=".\\data\\powerondrag.csv",  
    power_on_drag='.\\data\\powerondrag.csv',
    center_of_mass_without_motor=2.75, 
    coordinate_system_orientation="nose_to_tail", 
)
trb.add_motor(z4000, position=4.49)
# trb.all_info()

In [ ]:
#Aerodynamic surfaces taken from openrocket
nose_cone = trb.add_nose(
    length=0.7, kind="lvhaack", position=0
)

fin_set = trb.add_trapezoidal_fins(
    n=4,
    root_chord=0.287,
    tip_chord=0.059,
    span=0.202,
    sweep_length=0.228,
    position=4.21,
    cant_angle=0,
)

tail = trb.add_tail(
    top_radius=0.1, bottom_radius=0.065, length=0.287, position=4.21
)

rail_buttons = trb.set_rail_buttons(
    upper_button_position=2.17, #Just some value, not accurate
    lower_button_position=3.5, #Just some value, not accurate
    angular_position=0, #Just some value, not accurate #0 stopni
)

main = trb.add_parachute(
    name="main",
    cd_s=12.72, # cd * parachute area
    trigger=1000,      
    sampling_rate=105,
    lag=6,
    noise=(0, 8.3, 0.5),
    radius=2.25, 
    height=2.25,
    porosity=0.0432,
)

drogue = trb.add_parachute(
    name="drogue",
    cd_s=1.218,
    trigger="apogee", 
    sampling_rate=105,
    lag=1,
    noise=(0, 8.3, 0.5),
    radius=0.76,
    height=0.76,
    porosity=0.0432,
)

# trb.draw()
# trb.plots.drag_curves()

In [ ]:
test_flight = Flight(
    rocket=trb, environment=env, rail_length=15.24, inclination=85, heading=0
    )

#trb.all_info()
#test_flight.all_info()
#test_flight.plots.trajectory_3d()


In [50]:
#Rail Departure Velocity

# 1. Get SI Value from RocketPy
v_m_s = test_flight.out_of_rail_velocity

# 2. Convert to Imperial (ft/s)
v_ft_s = v_m_s * 3.28084

# 3. Get the exact time of rail exit
t_exit = test_flight.out_of_rail_time

print(f"Rail Departure Velocity (SI): {v_m_s:.2f} m/s")
print(f"Rail Departure Velocity (Imp): {v_ft_s:.2f} ft/s")
print(f"Time of Rail Exit: {t_exit:.3f} s")

Rail Departure Velocity (SI): 31.68 m/s
Rail Departure Velocity (Imp): 103.92 ft/s
Time of Rail Exit: 1.804 s


In [ ]:
#Average TWR Calculation (Rail Phase)

t_exit = test_flight.out_of_rail_time 
g = 9.80665  # SI gravity 

#  100 sample points between 0 and t_exit 
rail_times = np.linspace(1, t_exit, 100)

# 3. Calculate Average Thrust 
thrust_values = [z4000.thrust(t) for t in rail_times]
avg_thrust = np.mean(thrust_values)

# 4. Calculate Average Mass during rail phase
propellant_mass_values = [z4000.propellant_mass(t) for t in rail_times]
total_mass_values = [trb.mass + z4000.propellant_mass(t) for t in rail_times]
avg_propellant_mass = np.mean(propellant_mass_values)
avg_total_mass = trb.mass + avg_propellant_mass # trb.mass is dry mass

# 5. Calculate TWR
twr_rail_avg = avg_thrust / (avg_total_mass * g)

# --- OUTPUT ---
print(f"Time on Rail: {t_exit:.4f} s")
print(f"Average Thrust (Rail Phase): {avg_thrust:.2f} N")
print(f"Average Total Mass (Rail Phase): {avg_total_mass:.2f} kg")
print(f"---------------------------------------------------")
print(f"TWR (Rail Phase Average): {twr_rail_avg:.4f}")

In [ ]:
test_flight.prints.stability_margin()


Stability Margin

Initial Stability Margin: 2.871 c at 0.00 s
Out of Rail Stability Margin: 2.862 c at 1.80 s
Maximum Stability Margin: 3.818 c at 12.25 s
Minimum Stability Margin: 2.862 c at 1.77 s


In [ ]:

import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

# 1. Get Angle of Attack Data
# Fix: Extract only the 2nd column (values) if the array is 2D
raw_alpha = np.array(test_flight.angle_of_attack)
if raw_alpha.ndim > 1:
    alpha_values = raw_alpha[:, 1]  # Column 1 is Angle, Column 0 is Time
    time_values = raw_alpha[:, 0]
else:
    alpha_values = raw_alpha
    time_values = np.array(test_flight.time)

# 2. Filter for Ascent Phase (Rail Exit -> Apogee)
t_exit = test_flight.out_of_rail_time
t_apogee = test_flight.apogee_time
mask = (time_values > t_exit + 0.5) & (time_values < t_apogee - 1.0) # Buffer to avoid noise

t_data = time_values[mask]
alpha_data = alpha_values[mask]

# 3. Find Peaks in the Oscillation (Local Maxima)
peaks, _ = find_peaks(alpha_data, distance=5) 
peak_times = t_data[peaks]
peak_values = alpha_data[peaks]

# 4. Calculate Damping Ratio (Zeta) using Logarithmic Decrement
damping_ratios = []
damping_times = []

for i in range(len(peak_values) - 1):
    A1 = peak_values[i]
    A2 = peak_values[i+1]
    
    # Only calculate if amplitude is decaying and significant
    if A1 > A2 and A1 > 0.5: 
        delta = np.log(A1 / A2)
        zeta = 1 / np.sqrt(1 + (2 * np.pi / delta)**2)
        damping_ratios.append(zeta)
        damping_times.append(peak_times[i])

# 5. Output Results for Table
if len(damping_ratios) == 0:
    print("No significant oscillations detected. Rocket is likely Overdamped (zeta > 1.0).")
else:
    min_zeta = np.min(damping_ratios)
    max_zeta = np.max(damping_ratios)
    t_min = damping_times[np.argmin(damping_ratios)]
    t_max = damping_times[np.argmax(damping_ratios)]

    print(f"=== Damping Ratio Results ===")
    print(f"Lowest Damping Ratio:  {min_zeta:.4f} at t={t_min:.2f} s")
    print(f"Highest Damping Ratio: {max_zeta:.4f} at t={t_max:.2f} s")
    
    # Validation
    if min_zeta < 0.05:
        print(f"⚠️  WARNING: Underdamped (< 0.05). Risk of instability.")
    else:
        print(f"✅ PASS: Minimum damping > 0.05.")

    # 6. Plot for Verification
    plt.figure(figsize=(10, 5))
    plt.plot(t_data, alpha_data, label='Angle of Attack (deg)')
    plt.plot(peak_times, peak_values, "x", color='red', label='Peaks')
    plt.xlabel("Time (s)")
    plt.ylabel("Alpha (deg)")
    plt.title("Angle of Attack Oscillations")
    plt.legend()
    plt.grid(True)
    plt.show()

In [52]:
# max acceleration, speed, number of mach, dynamic pressure
max_acceleration = test_flight.max_acceleration
max_speed = test_flight.max_speed
max_mach = test_flight.max_mach_number
max_dynamic_pressure = test_flight.max_dynamic_pressure
max_acceleration_time = test_flight.max_acceleration_time
max_speed_time = test_flight.max_speed_time
max_dynamic_pressure_time = test_flight.max_dynamic_pressure_time
altitude_at_max_pressure = test_flight.z(max_dynamic_pressure_time)
max_altitude = test_flight.apogee
max_altitude_time = test_flight.apogee_time
print(f"Maximum Acceleration: {max_acceleration:.2f} m/s^2")
print(f"Maximum Speed: {max_speed:.2f} m/s")
print(f"Maximum Mach Number: {max_mach:.2f}")
print(f"Maximum Dynamic Pressure: {max_dynamic_pressure:.2f} Pa")
print(f"Time of Maximum Acceleration: {max_acceleration_time:.2f} s")
print(f"Time of Maximum Speed: {max_speed_time:.2f} s")
print(f"Time of Maximum Dynamic Pressure: {max_dynamic_pressure_time:.2f} s")
print(f"Altitude at Maximum Dynamic Pressure: {altitude_at_max_pressure:.2f} m")
print(f"Maximum Altitude: {max_altitude:.2f} m")
print(f"Time of Maximum Altitude: {max_altitude_time:.2f} s")

Maximum Acceleration: 65.64 m/s^2
Maximum Speed: 290.15 m/s
Maximum Mach Number: 0.87
Maximum Dynamic Pressure: 39645.87 Pa
Time of Maximum Acceleration: 160.76 s
Time of Maximum Speed: 11.42 s
Time of Maximum Dynamic Pressure: 11.36 s
Altitude at Maximum Dynamic Pressure: 2556.85 m
Maximum Altitude: 5909.16 m
Time of Maximum Altitude: 36.43 s


In [53]:
# converting to imperial
max_acceleration_ft_s2 = max_acceleration * 3.28084
max_speed_ft_s = max_speed * 3.28084
max_dynamic_pressure_psf = max_dynamic_pressure * 0.0208854342
altitude_at_max_pressure_ft = altitude_at_max_pressure * 3.28084
max_altitude_ft = max_altitude * 3.28084
print(f"Maximum Acceleration: {max_acceleration_ft_s2:.2f} fts^2")
print(f"Maximum Speed: {max_speed_ft_s:.2f} ft/s")
print(f"Maximum Dynamic Pressure: {max_dynamic_pressure_psf:.2f} psf")
print(f"Altitude at Maximum Dynamic Pressure: {altitude_at_max_pressure_ft:.2f} ft")
print(f"Maximum Altitude: {max_altitude_ft:.2f} ft")

Maximum Acceleration: 215.36 fts^2
Maximum Speed: 951.94 ft/s
Maximum Dynamic Pressure: 828.02 psf
Altitude at Maximum Dynamic Pressure: 8388.63 ft
Maximum Altitude: 19387.00 ft


In [ ]:
#max distance from the pad nominal
#test_flight.all_info()
x_max = test_flight.x.x_array.max()
print(f"Maximum Horizontal Distance from Pad: {x_max:.2f} m")
y_max = test_flight.y.y_array.max()
print(f"Maximum Lateral Distance from Pad: {y_max:.2f} m")
x_array = test_flight.x.x_array
y_array = test_flight.y.y_array
max_distance = np.sqrt(x_array**2 + y_array**2).max()
print(f"Maximum Distance from Pad: {max_distance:.2f} m")

In [ ]:
# Max Distance from Pad (Ballistic)
# here we have to turn off the parachutes, just comment the lines where they are added to the rocket
x_max = test_flight.x.x_array.max()
print(f"Maximum Horizontal Distance from Pad: {x_max:.2f} m")
y_max = test_flight.y.y_array.max()
print(f"Maximum Lateral Distance from Pad: {y_max:.2f} m")
x_array = test_flight.x.x_array
y_array = test_flight.y.y_array
max_distance = np.sqrt(x_array**2 + y_array**2).max()
print(f"Maximum Distance from Pad: {max_distance:.2f} m")
test_flight.plots.trajectory_3d()

In [ ]:
# Max distance from the pad (drogue only)
# comment only main
x_max = test_flight.x.x_array.max()
print(f"Maximum Horizontal Distance from Pad: {x_max:.2f} m")
y_max = test_flight.y.y_array.max()
print(f"Maximum Lateral Distance from Pad: {y_max:.2f} m")
x_array = test_flight.x.x_array
y_array = test_flight.y.y_array
max_distance = np.sqrt(x_array**2 + y_array**2).max()
print(f"Maximum Distance from Pad: {max_distance:.2f} m")
test_flight.plots.trajectory_3d()

In [ ]:
# Max distance main at apogee
# comment drogue, main trigger apogee
x_max = test_flight.x.x_array.max()
print(f"Maximum Horizontal Distance from Pad: {x_max:.2f} m")
y_max = test_flight.y.y_array.max()
print(f"Maximum Lateral Distance from Pad: {y_max:.2f} m")
x_array = test_flight.x.x_array
y_array = test_flight.y.y_array
max_distance = np.sqrt(x_array**2 + y_array**2).max()
print(f"Maximum Distance from Pad: {max_distance:.2f} m")
test_flight.plots.trajectory_3d()

In [ ]:
# pitch/yaw moments
# Aerodynamic moments in body frame (N⋅m)
M1 = test_flight.M1      # Roll moment
M2 = test_flight.M2      # Pitch moment
M3 = test_flight.M3      # Yaw moment
M1_array = np.array(M1)
M2_array = np.array(M2)
M3_array = np.array(M3)
M1_column = M1_array[:, 1] 
M2_column = M2_array[:, 1]
M3_column = M3_array[:, 1]
M1_max = M1_column.max()
M2_max = M2_column.max()
M3_max = M3_column.max()
M1_max_time = M1_array[M1_column.argmax(), 0]
M2_max_time = M2_array[M2_column.argmax(), 0]
M3_max_time = M3_array[M3_column.argmax(), 0]
print(f"Max roll moment: {M1_max} N⋅m")
print(f"Max pitch moment: {M2_max} N⋅m")
print(f"Max yaw moment: {M3_max} N⋅m") 
print(f"Time of max roll moment: {M1_max_time:.2f} s")
print(f"Time of max pitch moment: {M2_max_time:.2f} s")
print(f"Time of max yaw moment: {M3_max_time:.2f} s") 
              

In [ ]:
trb.evaluate_center_of_dry_mass()
cp = trb.evaluate_center_of_pressure()
cp.savetxt("cp.csv", lower=0, upper=1, samples=20, fmt="%.4f")
com = trb.evaluate_center_of_mass()
com.savetxt("com.csv", lower=0, upper=1, samples=20, fmt="%.4f") 
f_mach = test_flight.mach_number
f_mach.savetxt("mach.csv", lower=0, upper=13, samples=20, fmt="%.4f")


In [ ]:
cp_outputs = np.loadtxt("cp.csv", delimiter=",", skiprows=1)
com_outputs = np.loadtxt("com.csv", delimiter=",", skiprows=1)  
f_mach_outputs = np.loadtxt("mach.csv", delimiter=",", skiprows=1)





In [ ]:
# step 1 we get functions for cp, com and mach number
cp = trb.evaluate_center_of_pressure()
com = trb.evaluate_center_of_mass()
mach = test_flight.mach_number

# step 2 we save them to csv
cp.savetxt("cp.csv", lower=0, upper=1, samples=100, fmt="%.4f")
com.savetxt("com.csv", lower=0, upper=14, samples=20, fmt="%.4f")
mach.savetxt("mach.csv")

# step 3 we load them back from csv as pd dataframes
cp_df = pd.read_csv("cp.csv", header=0)
com_df = pd.read_csv("com.csv", header=0)
mach_df = pd.read_csv("mach.csv", header=0) 

mach_unique_df = mach_df.drop_duplicates(subset='Mach Number')

mapping = mach_unique_df.set_index('Mach Number')['Time (s)']

cp_df['x'] = cp_df['x'].map(mapping).fillna(cp_df['x'])
cp_df.head()

In [ ]:
# 1. Load Data (as you did)
cp_df = pd.read_csv("cp.csv", header=0)     # Columns likely: x (Mach), y (CP)
mach_df = pd.read_csv("mach.csv", header=0) # Columns: Time (s), Mach Number
max_index = mach_df['Mach Number'].idxmax()
print(f"Max Mach Number: {mach_df.loc[max_index, 'Mach Number']} at Time: {mach_df.loc[max_index, 'Time (s)']} s")
mach_df = mach_df.loc[:max_index]
# 2. Rename columns for clarity (Optional but recommended)
# Let's assume cp_df 'x' is Mach
cp_df = cp_df.rename(columns={'x': 'Mach_Ref'}) 

# 3. SORTING IS MANDATORY for merge_asof
cp_df = cp_df.sort_values('Mach_Ref')
mach_df = mach_df.sort_values('Mach Number')

# 4. Perform the "Nearest" Match
# This looks for the closest 'Mach Number' in mach_df for every 'Mach_Ref' in cp_df
merged_df = pd.merge_asof(
    cp_df, 
    mach_df, 
    left_on='Mach_Ref', 
    right_on='Mach Number', 
    direction='nearest'
)

# 5. Result
# Now you have CP data alongside the Time it occurred (approximately)
# You can replace the 'x' column now if you really want to:
merged_df['x'] = merged_df['Time (s)']

print(merged_df.head())
plt.figure(figsize=(10, 6))

# 1. Plot Center of Mass (Time is already x-axis here)
plt.plot(
    com_df['Time (s)'], 
    com_df['Center of Mass Position (m)'], 
    label='Center of Mass (COM)', 
    color='#1565c0',  # Blue
    linewidth=2
)

# 2. Plot Center of Pressure (Use flight time as x-axis)
plt.plot(
    merged_df['Time (s)'], 
    merged_df['Scalar'], 
    label='Center of Pressure (CP)', 
    color='red', 
    linewidth=2
)

# Formatting
plt.title("Rocket Stability: COM & CP vs Time")
plt.xlabel("Time (s)")
plt.ylabel("Position from Nose Cone (m)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Save or Show
#plt.savefig("stability_over_time.png", dpi=300)
plt.show()

In [ ]:
angle_of_attack = test_flight.angle_of_attack
#angle_of_attack.savetxt("angle_of_attack.csv", lower=0, upper=14, samples=100, fmt="%.4f")
angle_of_attack_df = pd.read_csv("angle_of_attack.csv", header=0)